In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())

CUDA Available: False
Number of GPUs: 0


In [1]:
import pandas as pd
import lstm_functions as lf
import datetime as dt
import numpy as np
from scipy.ndimage import gaussian_filter

In [2]:
# oil_data = yf.download('CL=F', start='2023-01-01', end='2023-12-31')
# oil_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)

data = pd.read_csv('model/merged_marginal_data.csv')

data['Date'] = data.apply(
    lambda row: (
        dt.datetime(row['Year'].astype(int), row['Month'].astype(int), row['Day'].astype(int), 23, 59, 59)
        if row['Hour'].astype(int) == 24 or row['Hour'].astype(int) == 25
        else dt.datetime(row['Year'].astype(int), row['Month'].astype(int), row['Day'].astype(int), row['Hour'].astype(int))
    ), 
    axis=1
)

data.drop(['Year', 'Month', 'Day', 'Hour'], axis=1, inplace=True)
data.set_index('Date', inplace=True)
data = data['Price1']
data = data.sort_index()

In [3]:
data_series = pd.Series(data.values, index=pd.to_datetime(data.index))

epsilon = 1e-6
#avoid 0 values by small addition
data_series = data_series + epsilon

In [4]:
data_series.max()

np.float64(220.000001)

In [5]:
from sklearn.model_selection import KFold

# Data preparation: Ensure no duplicate indices
data_series = data_series[~data_series.index.duplicated(keep='first')]

# Gaussian-filtered data
gauss_data = pd.Series(gaussian_filter(data_series, sigma=1), index=data_series.index).astype(float)

# Define parameters
look_back = 42
transforms = [True, True]  # [log, difference]
lstm_params = {
    "nodes": [4, 8],  # Testing multiple values for nodes
    "epochs": [2, 10, 15],  # Testing multiple values for epochs
    "verbose": 0  # Constant verbose
}

transform_options = [
    [True, True],  # Log and difference
    [True, False],  # Log only
    [False, True],  # Difference only
    [False, False]  # No transformations
]

def nested_cross_validate_lstm(data_series, gauss_data, look_back, transform_options, lstm_params, outer_k=5, inner_k=3):
    """
    Perform nested cross-validation on LSTM model with raw and Gaussian-filtered data.

    Args:
        data_series (pd.Series): Original time series data.
        gauss_data (pd.Series): Gaussian-filtered time series data.
        look_back (int): Look-back period for LSTM.
        transform_options (list): List of transformation combinations [log, difference].
        lstm_params (dict): Dictionary of LSTM parameters to test.
        outer_k (int): Number of folds for the outer cross-validation.
        inner_k (int): Number of folds for the inner cross-validation.

    Returns:
        dict: Average RMSE for raw and Gaussian-filtered data with the best model parameters.
    """
    outer_kf = KFold(n_splits=outer_k, shuffle=True, random_state=1312)
    results = {
        "raw": [],
        "gaussian": []
    }

    # Outer cross-validation loop
    for outer_train_idx, outer_test_idx in outer_kf.split(data_series):
        train_raw, test_raw = data_series.iloc[outer_train_idx], data_series.iloc[outer_test_idx]
        train_gauss, test_gauss = gauss_data.iloc[outer_train_idx], gauss_data.iloc[outer_test_idx]

        # Inner cross-validation loop for hyperparameter tuning
        best_model_raw = {"rmse": float('inf'), "params": None, "transforms": None}
        best_model_gauss = {"rmse": float('inf'), "params": None, "transforms": None}
        inner_kf = KFold(n_splits=inner_k, shuffle=True, random_state=42)

        for inner_train_idx, inner_val_idx in inner_kf.split(train_raw):
            inner_train_raw, inner_val_raw = train_raw.iloc[inner_train_idx], train_raw.iloc[inner_val_idx]
            inner_train_gauss, inner_val_gauss = train_gauss.iloc[inner_train_idx], train_gauss.iloc[inner_val_idx]

            for transforms in transform_options:
                for nodes in lstm_params["nodes"]:
                    for epochs in lstm_params["epochs"]:
                        params = [nodes, epochs, lstm_params["verbose"]]

                        _, _, _, rmse_raw = lf.lstm_model(inner_train_raw, look_back, transforms, params)
                        if rmse_raw < best_model_raw["rmse"]:
                            best_model_raw["rmse"] = rmse_raw
                            best_model_raw["params"] = params
                            best_model_raw["transforms"] = transforms

                        _, pred_mean, pred_std, _ = lf.lstm_model(inner_train_gauss, look_back, transforms, params)
                        rmse_gauss = lf.gauss_compare(inner_train_raw, pred_mean, pred_std)
                        if rmse_gauss < best_model_gauss["rmse"]:
                            best_model_gauss["rmse"] = rmse_gauss
                            best_model_gauss["params"] = params
                            best_model_gauss["transforms"] = transforms

        # Train the best model on the full outer training data and evaluate on the outer test data
        best_params_raw = best_model_raw["params"]
        best_transforms_raw = best_model_raw["transforms"]
        best_params_gauss = best_model_gauss["params"]
        best_transforms_gauss = best_model_gauss["transforms"]

        # Raw data
        _, _, _, outer_rmse_raw = lf.lstm_model(train_raw, look_back, best_transforms_raw, best_params_raw)
        results["raw"].append(outer_rmse_raw)

        # Gaussian-filtered data
        _, out_pred_mean, out_pred_std, _ = lf.lstm_model(train_gauss, look_back, best_transforms_gauss, best_params_gauss)
        outer_rmse_gaussian = lf.gauss_compare(train_raw, out_pred_mean, out_pred_std)
        results["gaussian"].append(outer_rmse_gaussian)

    # Calculate average RMSE
    avg_rmse_raw = np.mean(results["raw"])
    avg_rmse_gaussian = np.mean(results["gaussian"])

    return {
        "raw": {"avg_rmse": avg_rmse_raw, "best_model": best_model_raw},
        "gaussian": {"avg_rmse": avg_rmse_gaussian, "best_model": best_model_gauss}
    }


# Execute nested cross-validation
cv_results = nested_cross_validate_lstm(data_series, gauss_data, look_back, transform_options, lstm_params, outer_k=5, inner_k=3)

# Output the results
print(f"Average RMSE (Raw Data): {cv_results['raw']['avg_rmse']:.3f}")
print(f"Best Model (Raw Data): {cv_results['raw']['best_model']}")
print(f"Average RMSE (Gaussian-Filtered Data): {cv_results['gaussian']['avg_rmse']:.3f}")
print(f"Best Model (Gaussian-Filtered Data): {cv_results['gaussian']['best_model']}")

# PLot best model
y_test, pred_mean, pred_std, rmse = lf.lstm_model(data_series, look_back, transforms, cv_results['raw']['best_model']['params'])
gauss_test, gauss_mean, gauss_std, gauss_rmse = lf.lstm_model(gauss_data, look_back, transforms, cv_results['gaussian']['best_model']['params'])

lf.plot_results(pred_mean, pred_std, gauss_mean, gauss_std, y_test)


C:\Users\tamas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


(4275, 1, 42)
(4275,)
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


C:\Users\tamas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


(4275, 1, 42)
(4275,)
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Test RMSE: 58.686


C:\Users\tamas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

(4275, 1, 42)
(4275,)
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


C:\Users\tamas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


(4275, 1, 42)
(4275,)
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Test RMSE: 65.515


C:\Users\tamas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

In [ ]:
#lstm predictions
look_back = 55
log = True
difference = True
transforms = [log, difference]

nodes = 4
epochs = 2
verbose = 1 # 0=print no output, 1=most, 2=less, 3=least
lstm_params = [nodes, epochs, verbose]

# Remove duplicate indices
data_series = data_series[~data_series.index.duplicated(keep='first')]

y_test, pred_mean, pred_std, rmse = lf.lstm_model(data_series, look_back, transforms, lstm_params)

In [ ]:
gauss_data = pd.Series(gaussian_filter(data_series, sigma=1), index=data_series.index).astype(float)
# running LSTM with Gaussian-filtered data
look_back = 55
log = True
difference = True
transforms = [log, difference]

nodes = 4
epochs = 50
verbose = 1 # 0=print no output, 1=most, 2=less, 3=least
lstm_params = [nodes, epochs, verbose]

# This compares the predicted values to the gaussian-filtered data
y_test, pred_mean, pred_std, rmse = lf.lstm_model(gauss_data, look_back, transforms, lstm_params)

## Gaussian-filtered predictions compared to the original data

In [ ]:
gaussian_compare = lf.gauss_compare(data_series, pred_mean, pred_std)

In [ ]:
#lstm predictions
look_back = 55
log = True
difference = True
transforms = [log, difference]

nodes = 4
epochs = 2
verbose = 1 # 0=print no output, 1=most, 2=less, 3=least
lstm_params = [nodes, epochs, verbose]

# Remove duplicate indices
data_series = data_series[~data_series.index.duplicated(keep='first')]

y_test, pred_mean, pred_std = lf.lstm_model(data_series, look_back, transforms, lstm_params)

In [ ]:
gauss_data = pd.Series(gaussian_filter(data_series, sigma=1), index=data_series.index).astype(float)
# running LSTM with Gaussian-filtered data
look_back = 55
log = True
difference = True
transforms = [log, difference]

nodes = 4
epochs = 50
verbose = 1 # 0=print no output, 1=most, 2=less, 3=least
lstm_params = [nodes, epochs, verbose]

# This compares the predicted values to the gaussian-filtered data
y_test, pred_mean, pred_std = lf.lstm_model(gauss_data, look_back, transforms, lstm_params)

## Gaussian-filtered predictions compared to the original data

In [ ]:
gaussian_compare = lf.gauss_compare(data_series, pred_mean, pred_std)